In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import h5py
import pandas as pd
import numpy as np

import rushd as rd

from pathlib import Path
import re

import itertools
sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
def dox_string_to_float(s):
    # Handle '0'
    if s == '0' or s == 0:
        return 0.0
    # Handle 10^-xxx
    if s.startswith('10^-'):
        return 10**(-1 * float(s[4:]))
    raise RuntimeError(f"Unable to convert dox string: '{s}' to float.")

In [ ]:
single_integration_df = rd.flow.load_csv_with_metadata(
    rd.datadir/'instruments'/'data'/'attune'/'chris'/'2022.04.03-TANGLES-CRISPR'/'csv_export',
    str(rd.datadir/'instruments'/'data'/'attune'/'chris'/'2022.04.03-TANGLES-CRISPR'/'csv_export'/'plate_layout.yaml'))
single_integration_df = single_integration_df[
    (single_integration_df['mRuby2-A'] > 10) &
    (single_integration_df['eGFP-A'] > 10)
].copy()
single_integration_df.dox = single_integration_df.dox.map(dox_string_to_float)

In [ ]:
g = sns.scatterplot(data=single_integration_df, y='mRuby2-A', x='eGFP-A', hue='condition', alpha=0.4, s=5, 
    palette={'AAVS TALEN': '#BA3815', 'Clybl TALEN': '#E07D61', 'CRISPR (iCAG)': '#84A98C', 'CRISPR (LCV2)': '#52796F', 'CRISPR (MGS7)': '#446461'},
    hue_order=['AAVS TALEN', 'Clybl TALEN', 'CRISPR (iCAG)', 'CRISPR (LCV2)', 'CRISPR (MGS7)'])
g.set(xscale='log', yscale='log')
plt.ylim([1e1, 3e4])
plt.xlim([1e1, 3e5])
sns.despine()
plt.gca().legend(bbox_to_anchor=(1.05,0.8))
plt.savefig(rd.rootdir/'img'/'single_integration_dots.png', bbox_inches='tight', dpi=400)
plt.show()

for c in ['Clybl TALEN']:
    g = sns.jointplot(
        data=single_integration_df[
            (single_integration_df.dox > .001) &
            (single_integration_df.condition == c) &
            (single_integration_df['mRuby2-A'] > 2e2) &
            (single_integration_df['eGFP-A'] > 2e2)],
        x='eGFP-A', y='mRuby2-A', hue='dox', log_scale=True, kind='kde', marginal_kws={'common_norm':False})
    #plt.xlim([1e1, 3e4])
    #plt.ylim([1e1, 3e5])
    plt.savefig(rd.rootdir/'img'/'clybl_talen_integration_results.svg', bbox_inches='tight')
    plt.show()